In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm


In [3]:
df = pd.read_csv("companydata_part1.csv")

/tmp/ipykernel_1952/3899070418.py:1: DtypeWarning: Columns (41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("companydata_part1.csv")


In [6]:
df["RegAddress.PostCode"].unique()

array(['HG1 1ND', 'EC1V 9LT', 'EN5 5LB', ..., 'OX12 7FP', 'GU46 6BX',
       'KA3 3DF'], shape=(326033,), dtype=object)

In [4]:
uk_countries = ["ENGLAND", "SCOTLAND", "WALES", "NORTHERN IRELAND", "UNITED KINGDOM"]

In [5]:
df_uk = df[df["RegAddress.Country"].isin(uk_countries)]

In [6]:
df_uk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 688497 entries, 0 to 849995
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   CompanyName                         688497 non-null  object 
 1    CompanyNumber                      688497 non-null  object 
 2   RegAddress.CareOf                   1403 non-null    object 
 3   RegAddress.POBox                    452 non-null     object 
 4   RegAddress.AddressLine1             688494 non-null  object 
 5    RegAddress.AddressLine2            391236 non-null  object 
 6   RegAddress.PostTown                 688491 non-null  object 
 7   RegAddress.County                   173133 non-null  object 
 8   RegAddress.Country                  688497 non-null  object 
 9   RegAddress.PostCode                 688233 non-null  object 
 10  CompanyCategory                     688497 non-null  object 
 11  CompanyStatus                  

We have now created our df_uk with only our addresses in the UK. We will be looking next to include smaller nations.



In [7]:
df_uk.DissolutionDate.unique()

array([nan])

Seems like all those companies are active



In [8]:
df_uk["AddressLine1"] = df["RegAddress.AddressLine1"].str.strip().str.upper()

/tmp/ipykernel_1952/4050367734.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["AddressLine1"] = df["RegAddress.AddressLine1"].str.strip().str.upper()


In [9]:
df_uk["AddressLine2"] = df[" RegAddress.AddressLine2"].str.strip().str.upper()

/tmp/ipykernel_1952/3541353132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["AddressLine2"] = df[" RegAddress.AddressLine2"].str.strip().str.upper()


In [10]:
df_uk["Town_clean"]= df["RegAddress.PostTown"].str.strip().str.upper()

/tmp/ipykernel_1952/2405946827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["Town_clean"]= df["RegAddress.PostTown"].str.strip().str.upper()


In [11]:
df_uk["RegAddress.County"] = df["RegAddress.County"].str.strip().str.upper()

/tmp/ipykernel_1952/2026710825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["RegAddress.County"] = df["RegAddress.County"].str.strip().str.upper()


In [12]:
df_uk["PostCode_clean"] = df_uk["RegAddress.PostCode"].str.upper().str.replace(" ", "").str.replace(r"^(.+)(\w{3})$", r"\1 \2", regex=True)

/tmp/ipykernel_1952/3621565947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["PostCode_clean"] = df_uk["RegAddress.PostCode"].str.upper().str.replace(" ", "").str.replace(r"^(.+)(\w{3})$", r"\1 \2", regex=True)


That was just cleaning the relevant columns for our data in case we want to use those

In [13]:
postcode_counts = (
    df_uk.groupby("PostCode_clean")["CompanyName"]
         .nunique()
         .reset_index(name="CompanyCount")
         .sort_values("CompanyCount", ascending=False)
)

In [14]:
# Companies per address (address line + postcode together)

address_counts = (
    df_uk.groupby(["PostCode_clean", "AddressLine1"])["CompanyName"]
         .nunique()
         .reset_index(name="Companies_at_Address")
)

# Companies per postcode (already cleaned)
postcode_counts = (
    df_uk.groupby("PostCode_clean")["CompanyName"]
         .nunique()
         .reset_index(name="Companies_in_Postcode")
)

# Merge the two for comparison
compare = address_counts.merge(postcode_counts, on="PostCode_clean", how="left")

# Add share of postcode that each address represents
compare["Address_share_in_PC"] = (
    (compare["Companies_at_Address"] / compare["Companies_in_Postcode"] * 100)
    .round(1)
    .astype(str) + "%"
)

# See the biggest hubs
compare.sort_values("Companies_at_Address", ascending=False).head(20)



,PostCode_clean,AddressLine1,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC
397632,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%
121264,EC1V 2NX,128 CITY ROAD,5795,9248,62.7%
229267,N1 7GU,20-22 WENLOCK ROAD,2699,5408,49.9%
229258,N1 7GU,20 WENLOCK ROAD,2622,5408,48.5%
388284,W1W 7LT,85 GREAT PORTLAND STREET,2311,2843,81.3%
396707,WC1N 3AX,27 OLD GLOUCESTER STREET,2055,2316,88.7%
121229,EC1V 2NX,124 CITY ROAD,1922,9248,20.8%
122013,EC2A 4NE,"3RD FLOOR, 86-90",1851,4041,45.8%
122049,EC2A 4NE,86-90 PAUL STREET,1464,4041,36.2%
388031,W1W 5PF,167-169 GREAT PORTLAND STREET,1265,1786,70.8%


In [15]:
# Heavy hitter addresses (building-level)
heavy_hitters = compare[compare["Companies_at_Address"] > 100].copy()

# Sort them by number of companies
heavy_hitters = heavy_hitters.sort_values("Companies_at_Address", ascending=False)

heavy_hitters.head(50)

,PostCode_clean,AddressLine1,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC
397632,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%
121264,EC1V 2NX,128 CITY ROAD,5795,9248,62.7%
229267,N1 7GU,20-22 WENLOCK ROAD,2699,5408,49.9%
229258,N1 7GU,20 WENLOCK ROAD,2622,5408,48.5%
388284,W1W 7LT,85 GREAT PORTLAND STREET,2311,2843,81.3%
396707,WC1N 3AX,27 OLD GLOUCESTER STREET,2055,2316,88.7%
121229,EC1V 2NX,124 CITY ROAD,1922,9248,20.8%
122013,EC2A 4NE,"3RD FLOOR, 86-90",1851,4041,45.8%
122049,EC2A 4NE,86-90 PAUL STREET,1464,4041,36.2%
388031,W1W 5PF,167-169 GREAT PORTLAND STREET,1265,1786,70.8%


In [16]:
# % of postcode already in compare (Address_share_in_PC)
# Now add % of all UK companies

total_companies = df_uk["CompanyName"].nunique()

heavy_hitters["Share_of_all_companies"] = (
    heavy_hitters["Companies_at_Address"] / total_companies * 100
).round(2)

In [17]:
heavy_hitters.to_csv("heavy_hitters_table.csv", index=False)


In [18]:
df_uk["FullAddress"] = (
    df_uk["AddressLine1"].fillna("") + ", " +
    df_uk["AddressLine2"].fillna("") + ", "
).str.replace(r"(,\s*)+", ", ", regex=True) \
 .str.replace(r"\s+", " ", regex=True) \
 .str.strip(", ")

/tmp/ipykernel_1952/2261810301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uk["FullAddress"] = (


In [21]:
heavy_hitters.head()

,PostCode_clean,AddressLine1,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,Share_of_all_companies
397632,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72
121264,EC1V 2NX,128 CITY ROAD,5795,9248,62.7%,0.84
229267,N1 7GU,20-22 WENLOCK ROAD,2699,5408,49.9%,0.39
229258,N1 7GU,20 WENLOCK ROAD,2622,5408,48.5%,0.38
388284,W1W 7LT,85 GREAT PORTLAND STREET,2311,2843,81.3%,0.34


In [19]:
full_addresses = df_uk[["PostCode_clean", "AddressLine1", "AddressLine2" "FullAddress"]].drop_duplicates()

KeyError: "['AddressLine2FullAddress'] not in index"

In [70]:
# Merge into heavy_hitters
heavy_hitters = heavy_hitters.merge(
    full_addresses,
    on=["PostCode_clean", "AddressLine1"],
    how="left"
)

In [71]:
heavy_hitters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   PostCode_clean         155 non-null    object
 1   AddressLine1           155 non-null    object
 2   Companies_at_Address   155 non-null    int64 
 3   Companies_in_Postcode  155 non-null    int64 
 4   Address_share_in_PC    155 non-null    object
 5   FullAddress            155 non-null    object
dtypes: int64(2), object(4)
memory usage: 7.4+ KB


In [74]:
heavy_hitters.


,PostCode_clean,AddressLine1,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,FullAddress
0,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,"71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
1,EC1V 2NX,128 CITY ROAD,5795,9248,62.7%,"128 CITY ROAD, LONDON, NAN EC1V 2NX"
2,N1 7GU,20-22 WENLOCK ROAD,2699,5408,49.9%,"20-22 WENLOCK ROAD, LONDON, NAN N1 7GU"
3,N1 7GU,20 WENLOCK ROAD,2622,5408,48.5%,"20 WENLOCK ROAD, LONDON, NAN N1 7GU"
4,W1W 7LT,85 GREAT PORTLAND STREET,2311,2843,81.3%,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON,..."
5,WC1N 3AX,27 OLD GLOUCESTER STREET,2055,2316,88.7%,"27 OLD GLOUCESTER STREET, LONDON, NAN WC1N 3AX"
6,EC1V 2NX,124 CITY ROAD,1922,9248,20.8%,"124 CITY ROAD, LONDON, NAN EC1V 2NX"
7,EC2A 4NE,"3RD FLOOR, 86-90",1851,4041,45.8%,"3RD FLOOR, 86-90, PAUL STREET, LONDON, ENGLAND..."
8,EC2A 4NE,86-90 PAUL STREET,1464,4041,36.2%,"86-90 PAUL STREET, LONDON, ENGLAND EC2A 4NE"
9,W1W 5PF,167-169 GREAT PORTLAND STREET,1265,1786,70.8%,"167-169 GREAT PORTLAND STREET, LONDON, NAN W1W..."


In [73]:
heavy_hitters[heavy_hitters.PostCode_clean == 'EC2A 4LU']

,PostCode_clean,AddressLine1,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,FullAddress
153,EC2A 4LU,4TH FLOOR,101,103,98.1%,"4TH FLOOR, 4 TABERNACLE STREET, LONDON, NAN EC..."


In [56]:
heavy_hitters["FullAddress"]

0      71-75 SHELTON STREET, LONDON, GREATER LONDON W...
1      71-75 SHELTON STREET, COVENT GARDEN, LONDON, N...
2             71-75 SHELTON STREET, LONDON, NAN WC2H 9JQ
3      71-75 SHELTON STREET, COVEN GARDEN, LONDON, NA...
4      71-75 SHELTON STREET, COVENT GARDEN, LONDON WC...
                             ...                        
901    4TH FLOOR, 4 TABERNACLE STREET, LONDON, NAN EC...
902            57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE
903            57A BROADWAY, LEIGH-ON-SEA, ESSEX SS9 1PE
904              57A BROADWAY, LEIGH-ON-SEA, NAN SS9 1PE
905              57A BROADWAY, LEIGH ON SEA, NAN SS9 1PE
Name: FullAddress, Length: 906, dtype: object

In [66]:
# Collapse multiple FullAddress variants into one (first non-null)
full_addresses = (
    df_uk.groupby(["PostCode_clean", "AddressLine1"])["FullAddress"]
         .first()
         .reset_index()
)

# Merge unique FullAddress into heavy_hitters
heavy_hitters = heavy_hitters.merge(
    full_addresses,
    on=["PostCode_clean", "AddressLine1"],
    how="left"
)


In [67]:
heavy_hitters

,PostCode_clean,AddressLine1,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,Share_of_all_companies,FullAddress_x,FullAddress_y,FullAddress
0,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
1,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
2,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
3,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
4,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
...,...,...,...,...,...,...,...,...,...
901,EC2A 4LU,4TH FLOOR,101,103,98.1%,0.01,"4TH FLOOR, 4 TABERNACLE STREET, LONDON, NAN EC...","4TH FLOOR, 4 TABERNACLE STREET, LONDON, NAN EC...","4TH FLOOR, 4 TABERNACLE STREET, LONDON, NAN EC..."
902,SS9 1PE,57A BROADWAY,101,101,100.0%,0.01,"57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE","57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE","57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE"
903,SS9 1PE,57A BROADWAY,101,101,100.0%,0.01,"57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE","57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE","57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE"
904,SS9 1PE,57A BROADWAY,101,101,100.0%,0.01,"57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE","57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE","57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE"


In [60]:
heavy_hitters = heavy_hitters.drop(columns=["FullAddress_x"]).rename(columns={"FullAddress_y": "FullAddress"})


In [61]:
heavy_hitters

,PostCode_clean,AddressLine1,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,Share_of_all_companies,FullAddress
0,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
1,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
2,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
3,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
4,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
...,...,...,...,...,...,...,...
901,EC2A 4LU,4TH FLOOR,101,103,98.1%,0.01,"4TH FLOOR, 4 TABERNACLE STREET, LONDON, NAN EC..."
902,SS9 1PE,57A BROADWAY,101,101,100.0%,0.01,"57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE"
903,SS9 1PE,57A BROADWAY,101,101,100.0%,0.01,"57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE"
904,SS9 1PE,57A BROADWAY,101,101,100.0%,0.01,"57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE"


In [64]:
# Step 1: collapse to one unique FullAddress per (postcode, address line 1)
full_addresses = (
    df_uk.groupby(["PostCode_clean", "AddressLine1"])["FullAddress"]
         .first()   # you could also use .mode()[0] for the most common variant
         .reset_index()
)

# Step 2: merge this unique mapping back into heavy_hitters
heavy_hitters = heavy_hitters.merge(
    full_addresses,
    on=["PostCode_clean", "AddressLine1"],
    how="left"
)

# Step 3: rename for clarity
heavy_hitters = heavy_hitters.rename(columns={"FullAddress": "CanonicalFullAddress"})



In [65]:
heavy_hitters

,PostCode_clean,AddressLine1,Companies_at_Address,Companies_in_Postcode,Address_share_in_PC,Share_of_all_companies,FullAddress_x,FullAddress_y
0,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
1,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
2,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
3,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
4,WC2H 9JQ,71-75 SHELTON STREET,11876,12062,98.5%,1.72,"71-75 SHELTON STREET, LONDON, GREATER LONDON W...","71-75 SHELTON STREET, LONDON, GREATER LONDON W..."
...,...,...,...,...,...,...,...,...
901,EC2A 4LU,4TH FLOOR,101,103,98.1%,0.01,"4TH FLOOR, 4 TABERNACLE STREET, LONDON, NAN EC...","4TH FLOOR, 4 TABERNACLE STREET, LONDON, NAN EC..."
902,SS9 1PE,57A BROADWAY,101,101,100.0%,0.01,"57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE","57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE"
903,SS9 1PE,57A BROADWAY,101,101,100.0%,0.01,"57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE","57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE"
904,SS9 1PE,57A BROADWAY,101,101,100.0%,0.01,"57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE","57A BROADWAY, LEIGH ON SEA, ESSEX SS9 1PE"


In [27]:
address_counts_og = (
    df_uk.groupby("AddressLine1")["CompanyName"]
         .nunique()
         .reset_index(name="CompanyCount")
         .sort_values("CompanyCount", ascending=False)
)

In [29]:
compare = address_counts.merge(postcode_counts, on="PostCode_clean", how="left")

# Add % share of postcode that each address accounts for
compare["Address_share_in_PC"] = (
    compare["Companies_at_Address"] / compare["Companies_in_Postcode"] * 100
)

# Sort by biggest addresses
compare = compare.sort_values("Companies_at_Address", ascending=False)

KeyError: 'PostCode_clean'

In [24]:
postcode_counts.head(3)

,PostCode_clean,CompanyCount
271739,WC2H 9JQ,12062
82830,EC1V 2NX,9248
156434,N1 7GU,5408


In [28]:
address_counts.head(3)

,AddressLine1,CompanyCount
237370,71-75 SHELTON STREET,11883
36958,128 CITY ROAD,5799
93145,20-22 WENLOCK ROAD,2703


In [23]:
heavy_hitters_PC = postcode_counts[postcode_counts["CompanyCount"] > 100]

print(f"{heavy_hitters_PC.shape[0]} postcodes have more than 100 companies registered.")
heavy_hitters_PC.head(3)

262 postcodes have more than 100 companies registered.


,PostCode_clean,CompanyCount
271739,WC2H 9JQ,12062
82830,EC1V 2NX,9248
156434,N1 7GU,5408


Our definition of heavy hitters will be addresses with more than 100 unique companies listed

In [ ]:
heavy_hitters_add = postcode_counts[postcode_counts["CompanyCount"] > 100]

print(f"{heavy_hitters_PC.shape[0]} postcodes have more than 100 companies registered.")
heavy_hitters_PC.head(3)

In [18]:
# total unique UK companies
companies_total = df_uk["CompanyName"].nunique()

In [19]:
# companies at heavy hitter postcodes
companies_in_hh_pc = (
    df_uk[df_uk["PostCode_clean"].isin(heavy_hitters_PC["PostCode_clean"])]
        ["CompanyName"]
        .nunique()
)


In [20]:
# percentage
pct_pc = companies_in_hh_pc / companies_total * 100

In [22]:
print(f"{companies_in_hh_pc:,} / {companies_total:,} UK companies "
      f"({pct_pc:.2f}%) are at heavy hitter postcodes (>100 companies).")

89,622 / 688,491 UK companies (13.02%) are at heavy hitter postcodes (>100 companies).


In [25]:
df_uk.head()

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,...,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,AddressLine1,PostCode_clean
0,! LTD,08209948,NaN,NaN,9 PRINCES SQUARE,NaN,HARROGATE,NaN,ENGLAND,HG1 1ND,...,NaN,NaN,NaN,NaN,NaN,NaN,25/09/2025,11/09/2024,9 PRINCES SQUARE,HG1 1ND
1,!BIG IMPACT GRAPHICS LIMITED,11743365,NaN,NaN,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,UNITED KINGDOM,EC1V 9LT,...,NaN,NaN,NaN,NaN,NaN,NaN,29/12/2025,15/12/2024,372 OLD STREET,EC1V 9LT
2,!NFLECTION ADVISORY LIMITED,15073164,NaN,NaN,52 MEADWAY,NaN,BARNET,NaN,UNITED KINGDOM,EN5 5LB,...,NaN,NaN,NaN,NaN,NaN,NaN,28/08/2025,14/08/2024,52 MEADWAY,EN5 5LB
3,!NFOGENIE LTD,13522064,NaN,NaN,71-75 SHELTON STREET,NaN,LONDON,GREATER LONDON,UNITED KINGDOM,WC2H 9JQ,...,NaN,NaN,NaN,NaN,NaN,NaN,03/08/2025,20/07/2024,71-75 SHELTON STREET,WC2H 9JQ
4,!NK MEDIA LIMITED,15801429,NaN,NaN,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON ...",FIRST FLOOR,LONDON,NaN,ENGLAND,W1W 7LT,...,NaN,NaN,NaN,NaN,NaN,NaN,16/06/2026,02/06/2025,"85 GREAT PORTLAND STREET, FIRST FLOOR, LONDON ...",W1W 7LT
